## Массивы

В python объекты в массивах хранятся в виде ссылок (указателей), сами объекты расположены в других местах.
Что в python является массивом? Списки, кортежи.

### Неизменяемость объектов

При попытке "изменить" объект (например, к строке добавить символы или к кортежу элементы) происходит создание нового объекта и соотвественно выделение под него памяти.

In [7]:
s = 'qwerty'
print(id(s))

139765984019120


In [8]:
s += 'asd'
print(id(s))

139765984030768


Создались 2 разных объекта.

## List vs Tuple

Главное отличие? Кортеж - неизменяемый массив, соответственно операции вставки и удаления недоступны.
Сложность поиска элемента О(n) - потому что нужно пройти все элементы, чтобы найти нужный (в худшем случае). Доступ по индексу за О(1), вместе с самим массивом в памяти хранится и его длина. Операции append и pop (без аргумента) - в среднем O(1) (потому что, зная длину массива, мы можем быстро получить доступ к последнему элементу), insert и pop (с аргументом) - O(n) (чтобы вставить элемент в начало (или удалить) нужно будет передвинуть все остальные элементы).

### Что больше занимает памяти кортеж из одного элемента или список?

In [15]:
import sys

In [16]:
sys.getsizeof([1])

80

In [17]:
sys.getsizeof((1,))

64

Больше памяти занимает список, так как под него происходит выделение дополнительной памяти (запас) - этот процесс называется аллокацией.

sys.getrefcount() - с помощью этого метода можно подсчитать количество ссылок в коде на конкретный объект.

### Выделение памяти для списков

Для того чтобы при каждом добавлении элемента не обращаться к памяти и не выделять дополнительное место, список создается не по количеству элементов, а с запасом. Например, под список из одного элемента выделится сразу 4 ячейки памяти, из 5 - 8 ячеек, из 9 - 16 ячеек и тд. Существует специальная формула расчета, чтобы определить сколько ячеек выделяется.

In [61]:
l = []
print(sys.getsizeof(l))

72


In [62]:
l.append(0)
print(sys.getsizeof(l))

104


In [63]:
l.append(1)
print(sys.getsizeof(l))

104


In [64]:
l.append(2)
print(sys.getsizeof(l))

104


In [65]:
l.append(3)
print(sys.getsizeof(l))

104


In [66]:
l.append(4)
print(sys.getsizeof(l))

136


In [67]:
print(l)

[0, 1, 2, 3, 4]


Для того чтобы ускорить процесс создания списка (если мы знаем сколько будет в нем элементов) можно сразу создать "скелет" списка и затем уже заполнять его значениями. Это эффективнее, чем поэлементное добавление в изначально пустой список.

In [78]:
%%timeit
l = []
for i in range(10000):
	l.append(i)

352 µs ± 3.31 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [79]:
%%timeit
l = [0] * 10000
for i in range(10000):
	l[i] = i

257 µs ± 17.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Если вы знаете заранее, какой тип элементов у вас будет храниться в списке, то можно воспользоваться типизированным массивом.

In [80]:
from array import array
arr = array('i', (1, )*1000)

In [81]:
sys.getsizeof(arr)

4064

In [82]:
l = [1]*1000

In [83]:
sys.getsizeof(l)

8072

Чем больше количество элементов, тем существеннее разница в памяти, занимаемой массивом. Поэтому такой массив выгодно использовать для хранения большого количества однотипных данных.

### Переиспользование кортежей

В CPython есть еще такой интересный момент, как реюз ненужных кортежей. Например, вы создали кортеж с тремя элементами внутри, затем он вам стал не нужен, вы его удалили, но сборщик мусора не освободит это место, оно останется зарезервированным за кортежем с тремя элементами и в следующий раз, когда вы создадите новый кортеж с таким же количеством элементов, его поместят на то же место, где был ваш старый кортеж. Это работает с кортежами до 20 элементов.

In [85]:
a = (1,2)
print(id(a))
del a
b = ('a',4)
print(id(b))

139705844022400
139705844022400


#### 2 пустых кортежа

Если вы создадите 2 пустых кортежа, то они будут ссылаться на один объект.

In [88]:
c = ()
d = ()
print(c is d)

True


Это относится ко всем неизменяемым типам в python. Ниже 2 переменные, которые ссылаются на один объект - число 3.

In [90]:
num1, num2 = 3, 3
print(num1 is num2)

True


Дальше мои пояснения заканчиваются, можно смотреть вебинар и там мы с этим разбираемся)

### Почему join эффективнее для конкатенации строк, чем + ?

In [46]:
strings = ['Жизнь', 'слишком', 'коротка,', 'программируй', 'на', 'Python']

In [47]:
def join_strs(strs):
	result = ''
	for s in strs:
	    result += ' ' + s
	return result[1:]

In [48]:
def join_strs_better(strs):
    return ' '.join(strs)

In [49]:
%timeit join_strs(strings)

560 ns ± 10.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [50]:
%timeit join_strs_better(strings)

154 ns ± 2.3 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Словари и множества

In [52]:
a = 4
b = (1, 2)
c = [1, 2, 4]

In [53]:
a.__hash__()

4

In [54]:
c.__hash__()

TypeError: 'NoneType' object is not callable

In [55]:
from collections import OrderedDict, defaultdict

In [56]:
od = OrderedDict([('a', 1)])

In [57]:
od['a']

1

In [58]:
dd = defaultdict(str)
dd

defaultdict(str, {})

In [59]:
dd[1]

''

In [60]:
dd

defaultdict(str, {1: ''})

In [61]:
fs = frozenset({1, 2, 3, 3})

In [62]:
fs

frozenset({1, 2, 3})

In [64]:
fs.add(5)

AttributeError: 'frozenset' object has no attribute 'add'

In [63]:
fs.__hash__()

-272375401224217160

In [1]:
from collections import Counter

In [2]:
string = 'assrrfbbdds'

In [3]:
c = Counter(string)

In [4]:
c

Counter({'a': 1, 's': 3, 'r': 2, 'f': 1, 'b': 2, 'd': 2})

In [5]:
c.update('shhuuu')

In [6]:
c

Counter({'a': 1, 's': 4, 'r': 2, 'f': 1, 'b': 2, 'd': 2, 'h': 2, 'u': 3})

### Матрицы

In [40]:
import random
rows = random.randint(1,5)
cols = random.randint(1,4)
matrix = [[random.randint(0,20) for _ in range(cols)] for _ in range(rows)]

In [41]:
matrix

[[20, 13, 9], [7, 2, 2], [5, 3, 17], [18, 1, 5]]

In [42]:
transponse = [[0 for n in range(rows)] for _ in range(cols)]

In [43]:
transponse

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

In [44]:
for i in range(rows):
	for j in range(cols):
		transponse[j][i] = matrix[i][j]

In [45]:
transponse

[[20, 7, 5, 18], [13, 2, 3, 1], [9, 2, 17, 5]]

### Стек и очередь

In [1]:
from collections import deque

In [2]:
d = deque()
d.append(2)
d.append(3)
d.appendleft(1)

In [3]:
d

deque([1, 2, 3])

In [4]:
d.pop()

3

In [5]:
d.popleft()

1

In [6]:
d

deque([2])

In [7]:
import heapq

In [8]:
h = []
heapq.heappush(h, (4, 'd'))
heapq.heappush(h, (1, 'a'))
heapq.heappush(h, (3, 'c'))
heapq.heappush(h, (2, 'b'))

In [9]:
h

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]

In [10]:
heapq.heappop(h)

(1, 'a')